In [12]:
import pandas as pd
import numpy as np

# Reward와 Trade 로드
reward = reward.sort_values(["account_id", "ts"])
trade  = trade.sort_values(["account_id", "ts"])

# 1) 각 account_id의 모든 trade 시점을 dict로 저장
trade_map = trade.groupby("account_id")["ts"].apply(list).to_dict()

records = []

for acc, rws in reward.groupby("account_id"):
    trade_times = trade_map.get(acc, [])
    
    for rw_time in rws["ts"]:
        # reward 이후의 모든 trade 중 첫 번째
        future_trades = [t for t in trade_times if t > rw_time]

        if len(future_trades) > 0:
            gap_hours = (future_trades[0] - rw_time).total_seconds() / 3600
        else:
            gap_hours = np.nan
        
        records.append([acc, rw_time, gap_hours])

gap_df = pd.DataFrame(records, columns=["account_id", "reward_ts", "gap_hours"])

# 72h & 336h 기준
summary = gap_df.groupby("account_id")["gap_hours"].median().reset_index()

summary["inactive_flag"] = (
    summary["gap_hours"].isna() |
    (summary["gap_hours"] > 336)  # 14일
)

print(summary)


      account_id    gap_hours  inactive_flag
0   A_06d5bd3da9     0.199515          False
1   A_0ad0445c5a   497.392624           True
2   A_0bfb0b4311   298.763979          False
3   A_0e72128b0d     4.396809          False
4   A_1a7c006b71   101.637491          False
5   A_1b6957bd93     0.768575          False
6   A_26ffea8fd9     3.521013          False
7   A_27fd29fd55    21.135168          False
8   A_2ba62f7b20   218.499040          False
9   A_2db64f326b  6349.275449           True
10  A_33849d660b    46.525339          False
11  A_359a23d0bd   221.390928          False
12  A_3943c4c480  1380.659784           True
13  A_3b4f58349f     4.442584          False
14  A_4081f88bb0   951.699725           True
15  A_40cfe40ac2   956.238459           True
16  A_48fbd03d61    11.761469          False
17  A_4943ad62b4     1.864924          False
18  A_507a86d368     0.093935          False
19  A_522c68dd44   937.572397           True
20  A_55021b4ae2   128.049442          False
21  A_59b9